In [1]:
import numpy as np
import math
from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes
import random

In [2]:
def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a%b)

# Integer factiorization 

**Euler’s Formula for $pq$.**  
Let $p$ and $q$ be distinct primes and let $g = gcd(p − 1, q − 1)$. Then  
$a^{(p−1)(q−1)/g} ≡ 1 \ (mod \ pq)$ for all $a$ satisfying $gcd(a, pq) = 1$.

In particular, if $p$ and $q$ are odd primes, then  
$a^{(p−1)(q−1)/2} ≡ 1 \ (mod \ pq)$ for all $a$ satisfying $gcd(a, pq) = 1$.

Let $p$ be a prime and let $e ≥ 1$ be an integer satisfying
$gcd(e, p−1) = 1$. $e$ has an inverse modulo $p − 1$, say  
$de ≡ 1 \ (mod \ p − 1)$.  
Then the congruence
$x^e ≡ c \ (mod \ p)$  
has the unique solution $x ≡ c^d \ (mod \ p)$.

Let p, q be distinct primes and let e ≥ 1 be an integer satisfying
$gcd(e, (p−1)(q-1)) = 1$. e has an inverse modulo (p − 1)(q-1), say  
$de ≡ 1 \ (mod \ (p − 1)(q-1))$.  
Then the congruence
$x^e ≡ c \ (mod \ pq)$  
has the unique solution $x ≡ c^d \ (mod \ pq)$.

# RSA 

In [3]:
def create_key_rsa(p, q, e):
    assert gcd(e, (p-1)*(q-1)) == 1
    N = p*q
    return N, e
def encrypt_rsa(m):
    assert m < N
    c = pow(m, e, N)
    return c
def decrypt_rsa(c, p, q, e, N):
    d = inverse(e, (p-1)*(q-1))
    assert (d * e) % ((p-1)*(q-1)) == 1
    m = pow(c, d, N)
    return m

In [4]:
p = 1223
q = 1987
e = 948047
N, e = create_key_rsa(p, q, e)
N, e

(2430101, 948047)

In [5]:
m = b'se'
#m = 1070777
m = bytes_to_long(m)
c = encrypt_rsa(m)
c

1850200

In [6]:
long_to_bytes(decrypt_rsa(c, p,q, e, N))

b'se'

# Primality and factorization

**The Prime Number Theorem**  
$\lim_{X→∞}{\cfrac{π(X)} {X/ ln(X)} = 1}$.

A randomly chosen number N has
probability $\frac 1 {ln(N)}$ of being prime.

## Primality testing

**Definition**  
Fix an integer $n$. We say that an integer $a$ is a **witness** for (the compositeness of) $n$ if 
* $a^n \not\equiv a(mod \ n)$

Idea : Fermat's theorem says that if $p$ is prime then $a^p \equiv a (mod \ p)$. So we negate it

Remark: $\exists n$ s.t $n$ is composite and $a ^ n \equiv a (mod \ n)$  
Ex: $561 = 3 * 11 * 17$ and $561$ has no witnesses

### Miller Rabin test

Let $p$ be an odd prime and write $p−1=2^kq$ with $q$ odd.Let $a$ be any number NOT divisible by $p$. Then one of the following two conditions is true:
* $a^q \equiv 1 (mod \ p)$
* One of $a^q,a^{2q},a^{4q},...,a^{2^{k−1}q}$ is congruent to −1 modulo p.

https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test

In [53]:
def miller_rabin(n, bound):
    '''returns Composite or Probably prime'''
    
    #check parity
    if not n & 1:
        return 'Composite'
    
    k = 0
    q = n-1
    while not q & 1:
        q = q>>1
        k+=1
    assert(pow(2, k)* q == n-1)
    
    for i in range(bound):
        #print(i)
        a = random.randint(2, n-2)
        x = pow(a, q, n)
        if x == 1 or x == n-1:
            continue
        for _ in range(q-1):
            x = pow(x, 2, n)
            if x == n-1:
                continue
        return "Composite"
    return "Probably prime"
    

In [54]:
miller_rabin(561, 20)

0


'Composite'

In [55]:
miller_rabin(172947529, 20)

0
1


'Composite'

Remark:  For a certain bound, there is a deterministic version

## Pollard p-1 factoring algorithm

We are presented with a number $N = pq$ and our task is to determine
the prime factors $p$ and $q$. Suppose that by luck or hard work or some other
method, we manage to find an integer $L$ with the property that

$p − 1$ divides $L$ and $q − 1$ does not divide $L$

choose random $a$
=> $p | a^L − 1$ and $q$ does not divide $a^L − 1$ with very high probability  
=> $p = gcd(a^L − 1, N)$.  

Works if $p-1$ is a product of small primes  => $p-1$ divides $n!$ for a not so large n    
if $gcd(a^{n!} − 1, N) != 1$ => we found a divisor of N



In [56]:
def pollard_facto(N, B):
    #you can choose a to be other values
    a = 2
    for j in range(2, B):
        a = pow(a, j, N) #a^(n+1)! = a^(n! * (n+1))
        d = gcd(a-1, N)
        if 1 < d and d < N:
            return d
    return -1

In [57]:
N = 13927189

In [58]:
pollard_facto(N, 15)

3823

In [59]:
N = 168441398857
pollard_facto(N, 55)

350437

## B smooth numbers

**Definition**.  
An integer n is called B-smooth if all of its prime factors are less
than or equal to B.

The function $ψ(X, B)$ counts B-smooth numbers,

$ψ(X, B) =$ Number of B-smooth integers $n$ such that $1 < n ≤ X$.

Let $L(X) = e^{\sqrt{(ln X)(ln ln X)}}$ let $N$ be a large integer, and set $B = L(N)1/√2$.  

(a) We expect to check approximately $L(N)^{√2}$ random numbers modulo $N$ in order to find $π(B)$ numbers that are B-smooth.  

(b) We expect to check approximately $L(N)^{√2}$ random numbers of the form $a2 \ (mod \ N)$ in order to find enough B-
smooth numbers to factor N.

In [8]:
import math

In [23]:
def check_B_smooth(X, B):
    biggest_fact = -1
    if X ==1 :
        return True
    
    #check for 2
    while(X % 2 == 0):
        biggest_fact = max(biggest_fact, 2)
        X = int(X/2)
        
    #check for the res
    for i in range(3, int(math.sqrt(X))+1, 2):
        while(X % i == 0):
            biggest_fact = max(biggest_fact, i)
            X = int(X/i)
    
    #if X_end is prime then X_end divides X
    if(X>2):
        biggest_fact = max(biggest_fact, X)
        
    return biggest_fact <= B



In [24]:
check_B_smooth(24, 5)

True

In [25]:
d = {}
B = 5
for i in range(1, 25):
    d[str(i)] = check_B_smooth(i, B)

In [26]:
d

{'1': True,
 '2': True,
 '3': True,
 '4': True,
 '5': True,
 '6': True,
 '7': False,
 '8': True,
 '9': True,
 '10': True,
 '11': False,
 '12': True,
 '13': False,
 '14': False,
 '15': True,
 '16': True,
 '17': False,
 '18': True,
 '19': False,
 '20': True,
 '21': False,
 '22': False,
 '23': False,
 '24': True}

# TO DO : index calculus method, factorization via difference of squares, sieves

In [1]:
def index_calculus_method():
    pass

# Quadratic residues

**Definition**  
We say that $a$ is a quadratic residue modulo $p$ if $a$ is a square modulo $p$ i.e.   
$\exists c$ s.t $c^2≡a(mod\ p)$

**Prop**  
Let $p$ be an odd prime number.
* QR·QR=QR mod p
* QR·NR=NR mod p
* NR·NR=QR mod p

$\left( \cfrac{a}{p} \right) =$ Legendre symbol $=  
\left\{\begin{align*} 
&1 \  \text{if a is a quadratic residue} \ mod p,  \\
&-1 \ \text{if a is a quadratic nonresidue} \ mod p, \\  
&0 \ \text{if} \ p | a  
\end{align*}
\right.
$


$\left\{\right\}$

## Goldwasser-Micali

**Problem**: 
Let $p$ and $q$ be (secret) prime numbers and let $N=pq$ be given. For a given integer $a$, determine whether $a$ is a square modulo $N$, i.e., determine whether there exists an integer $u$ satisfying $u^2≡a(mod\ N)$.

Key creation:
* choose secret primes $p, q$
* choose $a$ with $(\frac a p) = (\frac a q) = -1$
* Publish $N = pq$ and $a$

Encryption:
* Choose plaintext $m \in \{0, 1\}$
* Choose random $r$ with $1<r<N$
* compute $c = 
\left\{
\begin{align*}
&r^2 mod \ N \text{if} \ m = 0 \\
&ar^2 mod \ N \text{if} \ m = 1 
\end{align*}
\right.
$
* return $c$

Decryption: 
* $m = 
\left\{
\begin{align*}
&0 \ \text{if} \ \left(\frac c p \right) = 1\\
&1 \  \text{if} \ \left(\frac c p\right) = 0
\end{align*}
\right.
$

*Proof*

$\left(\frac c p \right) = 
\left\{
\begin{align*}
&\left( \frac {r^2} p \right) = \left( \frac r p  \right)^2 = 1 \ \text{if} m = 0 \\
&\left( \frac {ar^2} p \right) = \left( \frac a p  \right) \left( \frac r p  \right)^2 = \left( \frac a p  \right) = -1 \ \text{if} m = 1 \\
\end{align*}
\right.
$

*Disadvantages*: 1 bit at a time => slow af

In [18]:
def legendre_symbol(a, p):
    return pow(a, (p-1)//2, p)

In [27]:
def create_key():
    p = getPrime(128)
    q = getPrime(128)
    N = p*q
    while True:
        a = random.randint(N//4, N-1)
        if(legendre_symbol(a, p) == p-1 and legendre_symbol(a, q) == q-1):
            break
    return (p, q), (N, a)
    
    

In [28]:
def encrypt(m, N, a):
    r = random.randint(2, N-1)
    if(m == 0):
        return pow(r, 2, N)
    elif(m == 1):
        return (a * pow(r, 2, N)) % N

In [31]:
def decrypt(p, c):
    if(legendre_symbol(c, p) == 1):
        return 0
    elif(legendre_symbol(c, p) == p-1):
        return 1

In [32]:
(p, q), (N, a) = create_key()

In [35]:
c = encrypt(1, N, a)

In [36]:
decrypt(p, c)

1